### Bibliotecas

In [1]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm #loop melhor 
import re
import os


import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords



# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.parsing.preprocessing import STOPWORDS


# Wordcloud
from wordcloud import WordCloud

In [4]:
# Importando base de dados
#%%time
base = pd.read_csv('Valor_Scrap.csv')

In [5]:
base

,Data,Título,Texto,Url
0,2011-07-25 10:20:00,Focus: mercado aponta estabilidade em inflação...,SÃO PAULO - O mercado manteve a proj...,https://valor.globo.com/brasil/noticia/2011/07...
1,2011-07-25 10:30:00,Bolsas da Europa caem; dívida dos EUA e rating...,SÃO PAULO - Os investidores nas bols...,https://valor.globo.com/financas/noticia/2011/...
2,2011-07-25 11:00:00,"Seis meses depois, Furnas só trocou de presidente",SÃO PAULO - Símbolo das mudanças que...,https://valor.globo.com/politica/noticia/2011/...
3,2011-07-25 11:10:00,Lupatech fecha contratos para plataformas de p...,"SÃO PAULO - A Lupatech S.A, especial...",https://valor.globo.com/empresas/noticia/2011/...
4,2011-07-25 14:04:00,Homem que confessou massacre na Noruega ficará...,ÃO PAULO – O juiz Kim Heger determin...,https://valor.globo.com/mundo/noticia/2011/07/...
...,...,...,...,...
678611,2022-12-31 19:28:00,"Cláudio Castro, que apoiou Bolsonaro, será o ú...","O governador do Rio, Cláudio Castro ...",https://valor.globo.com/politica/noticia/2022/...
678612,2022-12-31 19:37:00,Lula ainda não decidiu sobre desfile em carro ...,Futuro ministro da Comunicação Socia...,https://valor.globo.com/politica/noticia/2022/...
678613,2022-12-31 19:51:00,Lula conversa com autoridades da Ucrânia e Rús...,O presidente eleito Luiz Inácio Lula...,https://valor.globo.com/politica/noticia/2022/...
678614,2022-12-31 20:23:00,MEC publica portaria regulamentando cursos de ...,O Ministério da Educação (MEC) acaba...,https://valor.globo.com/empresas/noticia/2022/...


## Passo 1 - Realizando as modificações  
#### 1) Tirar pontuações
#### 2) Colocar tudo em letra minúscula 
#### 3) Retirar espaços duplos que vão aparecer após a retirada de pontuações



In [6]:
%%time 

# Remover pontuações
def remove_punct(text):
    """Remove pontuações do texto"""
    return re.sub(r'[^\w\s]', '', text)

colunas = ['Título']
for j in tqdm(colunas):
    ## Colocando os textos em formato de string 
    base[f'{j}'] = base[f'{j}'].astype(str)
    
    ### Tirando as pontuações
    ## Deleto as pontuações e colocamos '' no lugar - apagando as pontuações e criando uma nova coluna com o texto tratado
    base[f'{j}'] = base[f'{j}'].map(lambda x: re.sub('[,\.!?-]', ' ', x))
    
    # Colocando o texto em letras minúsculas
    base[f'{j}'] = base[f'{j}'].map(lambda x: x.lower())
    
    # Remover pontuações + uma vez
    base[f'{j}'] = base[f'{j}'].apply(remove_punct)
    
    # Tirando os espaços
    base[f'{j}'] = base[f'{j}'].replace(r'\s+', ' ', regex=True)



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.39s/it]

Wall time: 7.4 s


## Passo 2 - Retirar Números e StopWords

### Números

In [8]:
### Retirando Números + Manter SP+RJ ###

colunas = ['Título']
for j in tqdm(colunas): 
    base[f'{j}'] = base[f'{j}'].replace(to_replace=r'\d', value='', regex=True)
    base[f'{j}'] = base[f'{j}'].str.replace('são paulo', 'são_paulo')
    base[f'{j}'] = base[f'{j}'].str.replace('rio de janeiro', 'rio_de_janeiro')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.10s/it]


### StopWords - Parte 1

In [9]:
nltk.download('stopwords')
stop_words = stopwords.words('portuguese')
stop_words.extend(['reuters','valor', 'foto', 'foto:', 'reprodução'])


# Parte 1 - Retirando mais algumas palavras
stop_words.extend(['de','a','o','que','e','do','da','em','um','para','e','com','nao','uma','os','no','se','na','por','mais','as','dos','como','mas','foi',
                   'ao','ele','das','tem','a','seu','sua','ou','ser','quando','muito','ha','nos','ja','esta','eu','tambem','so','pelo','pela','ate','isso',
                   'ela','entre','era','depois','sem','mesmo','aos','ter','seus','quem','nas','me','esse','eles','estao','voce','tinha','foram','essa','num',
                   'nem','suas','meu','as','minha','tem','numa','pelos','elas','havia','seja','qual','sera','nos','tenho','lhe','deles','essas','esses','pelas',
                   'este','fosse','dele','tu','te','voces','vos','lhes','meus','minhas','teu','tua','teus','tuas','nosso','nossa','nossos','nossas','dela','delas',
                   'esta','estes','estas','aquele','aquela','aqueles','aquelas','isto','aquilo','estou','esta','estamos','estao','estive','esteve','estivemos',
                   'estiveram','estava','estavamos','estavam','estivera','estiveramos','esteja','estejamos','estejam','estivesse','estivessemos','estivessem',
                   'estiver','estivermos','estiverem','hei','ha','havemos','hao','houve','houvemos','houveram','houvera','houveramos','haja','hajamos','hajam',
                   'houvesse','houvessemos','houvessem','houver','houvermos','houverem','houverei','houvera','houveremos','houverao','houveria','houveriamos',
                   'houveriam','sou','somos','era','eramos','eram','fui','foi','fomos','foram','fora','foramos','seja','sejamos','sejam','fosse','fossemos',
                   'fossem','for','formos','forem','serei','sera','seremos','serao','seria','seriamos','seriam','tenho','tem','temos','tem','tinha','tinhamos',
                   'tinham','tive','teve','tivemos','tiveram','tivera','tiveramos','tenha','tenhamos','tenham','tivesse','tivessemos','tivessem','tiver','tivermos',
                   'tiverem','terei','tera','teremos','terao','teria','teriamos','teriam'])

# Parte 2 - Parte anterior com acento
stop_words.extend(['de',  'a',  'o',  'que',  'e',  'do',  'da',  'em',  'um',  'para',  'é',  'com',  'não',  'uma',  'os',  'no',  'se',  'na',  'por',  'mais',  
                    'as',  'dos',  'como',  'mas',  'foi',  'ao',  'ele',  'das',  'tem',  'à',  'seu',  'sua',  'ou',  'ser',  'quando',  'muito',  'há',  'nos',  
                    'já',  'está',  'eu',  'também',  'só',  'pelo',  'pela',  'até',  'isso',  'ela',  'entre',  'era',  'depois',  'sem',  'mesmo',  'aos',  'ter', 
                    'seus',  'quem',  'nas',  'me',  'esse',  'eles',  'estão',  'você',  'tinha',  'foram',  'essa',  'num',  'nem',  'suas',  'meu',  'às',  'minha',
                    'têm',  'numa',  'pelos',  'elas',  'havia',  'seja',  'qual',  'será',  'nós',  'tenho',  'lhe',  'deles',  'essas',  'esses',  'pelas',  'este',  
                    'fosse',  'dele',  'tu',  'te',  'vocês',  'vos',  'lhes',  'meus',  'minhas',  'teu',  'tua',  'teus',  'tuas',  'nosso',  'nossa',  'nossos',  
                    'nossas',  'dela',  'delas',  'esta',  'estes',  'estas',  'aquele',  'aquela',  'aqueles',  'aquelas',  'isto',  'aquilo',  'estou',  'está',  
                    'estamos',  'estão',  'estive',  'esteve',  'estivemos',  'estiveram',  'estava',  'estávamos',  'estavam',  'estivera',  'estivéramos',  'esteja',  
                    'estejamos',  'estejam',  'estivesse',  'estivéssemos',  'estivessem',  'estiver',  'estivermos',  'estiverem',  'hei',  'há',  'havemos',  'hão', 
                    'houve',  'houvemos',  'houveram',  'houvera',  'houvéramos',  'haja',  'hajamos',  'hajam',  'houvesse',  'houvéssemos',  'houvessem',  'houver',  
                    'houvermos',  'houverem',  'houverei',  'houverá',  'houveremos',  'houverão',  'houveria',  'houveríamos',  'houveriam',  'sou',  'somos',  'são', 
                    'era',  'éramos',  'eram',  'fui',  'foi',  'fomos',  'foram',  'fora',  'fôramos',  'seja',  'sejamos',  'sejam',  'fosse',  'fôssemos',  'fossem', 
                    'for',  'formos',  'forem',  'serei',  'será',  'seremos',  'serão',  'seria',  'seríamos',  'seriam',  'tenho',  'tem',  'temos',  'tém',  'tinha',  
                    'tínhamos',  'tinham',  'tive',  'teve',  'tivemos',  'tiveram',  'tivera',  'tivéramos',  'tenha',  'tenhamos',  'tenham',  'tivesse', 
                    'tivéssemos',  'tivessem',  'tiver',  'tivermos',  'tiverem',  'terei',  'terá',  'teremos',  'terão',  'teria',  'teríamos',  'teriam'])

# Parte 2 - Adicionando mais um conjunto de palavras, porém essas palavras possuem acentos.

const_words = [ 'a', 'à', 'adeus', 'agora', 'aí', 'ainda', 'além', 'algo', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ali', 'ampla', 'amplas', 'amplo', 
               'amplos', 'ano', 'anos', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 
               'área', 'as', 'às', 'assim', 'até', 'atrás', 'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze',
               'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contra', 'contudo', 'custa', 'da', 'dá',
               'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 
               'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 
               'deveriam', 'devia', 'deviam', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 
               'dizer', 'do', 'dois', 'dos', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'enquanto', 'entre', 'era', 'eram', 'éramos',
               'és', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estás', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 
               'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 
               'estivessem', 'estivéssemos', 'estiveste', 'estivestes', 'estou', 'etc', 'eu', 'exemplo', 'faço', 'falta', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos',
               'fazendo', 'fazer', 'fazes', 'feita', 'feitas', 'feito', 'feitos', 'fez', 'fim', 'final', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 
               'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foste', 'fostes', 'fui', 'geral', 'grande', 'grandes', 'grupo', 'há', 'haja', 'hajam', 'hajamos', 'hão',
               'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 
               'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'la', 'lá', 
               'lado', 'lhe', 'lhes', 'lo', 'local', 'logo', 'longe', 'lugar', 'maior', 'maioria', 'mais', 'mal', 'mas', 'máximo', 'me', 'meio', 'menor', 'menos', 'mês',
               'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'mil', 'minha', 'minhas', 'momento', 'muita', 'muitas', 'muito', 'muitos', 'na', 'nada', 'não', 
               'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'nem', 'nenhum', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 
               'ninguém', 'nível', 'no', 'noite', 'nome', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 
               'número', 'nunca', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 
               'para', 'parece', 'parte', 'partir', 'paucas', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 
               'pode', 'pude', 'pôde', 'podem', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'põe', 'põem', 'pois', 'ponto', 'pontos', 'por', 'porém',
               'porque', 'porquê', 'posição', 'possível', 'possivelmente', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros',
               'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'puderam', 'quais', 'quáis', 'qual', 'quando', 'quanto', 
               'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quê', 'quem', 'quer', 'quereis', 'querem', 'queremas', 'queres', 'quero', 'questão', 'quinta', 'quinto',
               'quinze', 'relação', 'sabe', 'sabem', 'se', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sejamos', 'sem', 'sempre', 'sendo', 'ser', 'será', 
               'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'sétima', 'sétimo', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 
               'só', 'sob', 'sobre', 'sois', 'somos', 'sou', 'sua', 'suas', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tanto', 'tão', 'tarde', 'te', 'tem',
               'tém', 'têm', ' só ', 'temos', 'tendes', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos',
               'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'ti', 'tido', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 
               'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes', 'toda', 'todas', 'todavia', 'todo', 'todos', 'trabalho', 'três', 'treze', 
               'tu', 'tua', 'tuas', 'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'vai', 'vais', 'vão', 'vários', 'vem', 'vêm', 'vendo',
               'vens', 'ver', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'vir', 'você', 'vocês', 'vos', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero']
stop_words.extend(const_words)

# Parte 3 - Algumas outras
stop_words.extend(['durante', 'pessoa', 'pergunte', 'perguntei', 'perguntando', 'pergunta','caso', 'sentido', 'deu', 'somente'])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gcard\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# função para remover palavras da lista
def remove_words(text, remove_list):
    return ' '.join([word for word in text.split() if word not in remove_list])

In [11]:
%%time

# Retirando as stopwords de cada uma das colunas
colunas = ['Título']
for j in tqdm(colunas): 
    base[f'{j}'] = base[f'{j}'].apply(lambda x: remove_words(x, stop_words))


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:13<00:00, 73.28s/it]

Wall time: 1min 13s


### StopWords - Parte 2

Foi obersado que **143** notícias estão em inglês.

*base[base['Texto'].str.startswith('the')]*

Vamos utilizar uma pequena lista de stopwords em inglês, já que 143 é um número baixo.

Fonte: Está nos comentários de https://gist.github.com/sebleier/554280

In [12]:
%%time

## Outro conjunto de stopwords em pt-br
stop_words2 = ['apesar', 'vamos']


# StopWords em Inglês
stop_words2.extend(["me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", 
                    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", 
                    "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", 
                    "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", 
                    "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", 
                    "having", "do", "does", "did", "doing", "an", "the", "and", "but", "if", 
                    "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", 
                    "about", "against", "between", "into", "through", "during", "before", "after", 
                    "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", 
                    "under", "again", "further", "then", "once", "here", "there", "when", "where",
                    "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", 
                    "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", 
                    "very", "can", "will", "just", "don", "should", "now"])


# Retirando as stopwords2 de cada uma das colunas
colunas = ['Título']
for j in tqdm(colunas): 
    base[f'{j}'] = base[f'{j}'].apply(lambda x: remove_words(x, stop_words2))




100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.74s/it]

Wall time: 7.77 s


In [13]:
# Tirando os espaços duplos

colunas = ['Título']
for j in tqdm(colunas):      
    base[f'{j}'] = base[f'{j}'].replace(r'\s+', ' ', regex=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.63s/it]


### Remover palavras com 1 letra e outras pontuações

In [14]:
def remove_one_letter_words(text):
    return re.sub(r'\b\w{1}\b', '', text)

colunas = ['Título']
for j in tqdm(colunas): 
    ## Retirando outras pontuações que ainda estavam no texto
    base[f'{j}'] = base[f'{j}'].apply(lambda x: re.sub('[ºª°]', '', x))
    ## Retirando palavras com uma letra
    base[f'{j}'] = base[f'{j}'].apply(remove_one_letter_words)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.74s/it]


In [15]:
base.head(3)

,Data,Título,Texto,Url
0,2011-07-25 10:20:00,focus mercado aponta estabilidade inflação,SÃO PAULO - O mercado manteve a proj...,https://valor.globo.com/brasil/noticia/2011/07...
1,2011-07-25 10:30:00,bolsas europa caem dívida eua rating grécia pr...,SÃO PAULO - Os investidores nas bols...,https://valor.globo.com/financas/noticia/2011/...
2,2011-07-25 11:00:00,furnas trocou presidente,SÃO PAULO - Símbolo das mudanças que...,https://valor.globo.com/politica/noticia/2011/...


# Base com Título e Data

In [19]:
base = base.drop(['Texto', 'Url'], axis=1)

In [20]:
base

,Data,Título
0,2011-07-25 10:20:00,focus mercado aponta estabilidade inflação
1,2011-07-25 10:30:00,bolsas europa caem dívida eua rating grécia pr...
2,2011-07-25 11:00:00,furnas trocou presidente
3,2011-07-25 11:10:00,lupatech fecha contratos plataformas produção ...
4,2011-07-25 14:04:00,homem confessou massacre noruega ficará detido...
...,...,...
678611,2022-12-31 19:28:00,cláudio castro apoiou bolsonaro único governad...
678612,2022-12-31 19:37:00,lula decidiu desfile carro aberto passagem fai...
678613,2022-12-31 19:51:00,lula conversa autoridades ucrânia rússia defen...
678614,2022-12-31 20:23:00,mec publica portaria regulamentando cursos med...


In [620]:
base.to_csv('Base_Final_Título.csv', index=False)

# Base com Data + Título e Primeiro Paragráfo da Notícia

Neste ponto, unimos o título e o primeiro parágrafo de cada notícia em uma única seção para fácil referência.

In [621]:
# Concatenar duas colunas 
base = base.assign(total=base['Título'].str.cat(base['pp'], sep=' '))  

In [622]:
base_final_total = base.loc[:, ['Data', 'total']]

In [623]:
base_final_total.to_csv('Base_Final_PP.csv', index=False)